# Quartisches Potential

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

Im Gegensatz zu den bisher betrachteten Anfangswertproblemen soll nun die Bewegung in einem quartischen Potential als Randwertproblem behandelt werden. Hierfür die SciPy-Funktion `integrate.solve_bvp` herangezogen.

Das dimensionslose Potential lautet

$$V(x) = \frac{1}{2} x^2 - \frac{1}{4} x^4 \,$$

so dass die Bewegungsgleichung durch

$$\frac{\text{d}^2x}{\text{d}t^2} = -x + x^3 $$

gegeben ist.

## Importanweisungen

Zusätzlich zu den bereits bekannten Importanweisungen wird hier die `HTML`-Klasse aus `IPython.display` importiert. Mit ihrer Hilfe wird später die Konvergenzinformation farblich dargestellt. Die Funktion `display`, die wir weiter unten benutzen werden, wird von aktuellen Versionen von `IPython` zur Verfügung gestellt, ohne dass ein weiterer Import erforderlich wäre.

In [ ]:
import numpy as np
from scipy import integrate
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import HTML
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

## Implementierung der Differentialgleichung

Die Bewegungsgleichung wird durch zwei Differentialgleichungen 1. Ordnung

\begin{align}
\frac{\text{d}x}{\text{d}t} &= v\\
\frac{\text{d}v}{\text{d}t} &= -x + x^3
\end{align}

dargestellt. Das Argument `t` ist von `integrate.solve_bvp` so vorgesehen, wird für die Rechnung jedoch nicht benötigt.

In [ ]:
def dx_dt(t, x_vec):
    x, v = x_vec
    a = -x + x**3
    return v, a

## Randbedingungen

Die gesuchte Lösung soll $x(0)=x(t_\text{end})=0$ erfüllen.

In [ ]:
def bc(x_vec_a, x_vec_b):
    x_a, v_a = x_vec_a
    x_b, v_b = x_vec_b
    return x_a, x_b

## Berechnung der Lösung des Randwertproblems

Die Funktion `integrate.solve_bvp` benötigt neben der Form der Differentialgleichung, die über `dx_dt` zur Verfügung gestellt wird, sowie den durch die Funktion `bc` definierten Randbedingungen und den Zeitpunkten `t` eine Startlösung für Ort und Geschwindigkeit. Da die Lösung bei $x(0)=0$ beginnen und bei $x(t_\text{end})=0$ enden soll, wählen wir für den Ort eine Sinusfunktion und für die Geschwindigkeit eine Kosinusfunktion mit einer vorgegebenen Anzahl `n_half_periods` von Halbperioden.

Um den Vergleich der Startlösung `x_init` und `v_init` mit der numerisch gefundenen Lösung `x` und `v` zu erlauben, werden diese Daten zusammen mit den Zeitpunkten `t` übergeben. Das Attribut `solution.success` enthält einen Wahrheitswert, der angibt, ob die Lösung konvergierte oder nicht.

In [ ]:
def trajectory(t_end, n_out, n_half_periods):
    omega = n_half_periods*np.pi/t_end
    t = np.linspace(0, t_end, n_out)
    x_init = np.sin(omega*t)
    v_init = np.cos(omega*t)
    solution = integrate.solve_bvp(dx_dt, bc, t,
                                   [x_init, v_init])
    x, v = solution.sol(t)
    return t, x, v, x_init, v_init, solution.success

## Implementierung der Bedienelemente und graphische Darstellung der Lösung

Mit den Schiebereglern lassen sich die folgenden Parameter einstellen:
- `t_end`: Länge des Zeitintervalls
- `n_half_periods`: Anzahl der Halbperioden in der gesuchten Lösung

Die Zahl der Datenpunkte ist in der Funktion `plot_result` vorgegeben und kann dort bei Bedarf geändert werden.

Ob die Lösung konvergierte oder nicht, wird in einem Farbbalken angegeben. Es werden sowohl die Startfunktionen als auch die numerisch erhaltenen Lösungen für Ort und Geschwindigkeit graphisch dargestellt. Damit lässt sich die Abweichung von Lösungen mit größerer Amplitude von einer harmonischen Schwingung deutlich machen. 

In [ ]:
widget_dict = {"t_end":
               widgets.FloatSlider(
                   value=22, min=1, max=40, step=1,
                   description=r"$t_\text{end}$"),
               "n_half_periods":
               widgets.IntSlider(
                   value=1, min=1, max=7, step=1,
                   description=r"$n_\text{half periods}$")
               }

out = widgets.Output()

@interact(**widget_dict)
def plot_result(t_end, n_half_periods):
    display(out)

    n_out = 200
    (t_values, x_values, v_values,
     x_init, v_init, success
     ) = trajectory(t_end, n_out, n_half_periods)

    with out:
        out.clear_output()
        if success:
            bgcolor = "#080"
            msg = "konvergierte Lösung"
        else:
            bgcolor = "#800"
            msg = "Lösung hat nicht konvergiert"
        style = (f"'color:#fff;background-color:{bgcolor};"
                 "padding:5px;'")
        display(HTML(f"<div style={style}>{msg}</div>"))

    fig, ax = plt.subplots()
    ax.plot(t_values, x_values, label="$x(t)$")
    ax.plot(t_values, v_values, label="$v(t)$")
    ax.plot(t_values, x_init,
            label=r"$x_\mathrm{init}(t)$")
    ax.plot(t_values, v_init,
            label=r"$v_\mathrm{init}(t)$")
    ax.set_xlabel("$t$")
    ax.set_ylabel("$x$")
    ax.legend(bbox_to_anchor=(1.02, 1), loc="upper left")